In [2]:
import os
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import urllib.parse
from datetime import datetime
import time
import pyperclip
import pyautogui
import random

options = Options()
options.add_argument('user-agent=' + "user-agent header")
options.add_argument('--mute-audio')
options.add_argument("--no-sandbox")
options.add_experimental_option("detach", True)
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
driver.get(url) 

driver.execute_script("document.getElementsByName('id')[0].value='id'")
time.sleep(1)
driver.execute_script("document.getElementsByName('pw')[0].value='pw'")
time.sleep(1)

login_btn = driver.find_element(By.ID,'log.login') 
login_btn.click() 
time.sleep(2)

def move_page(page):
    search_url = "https://cafe.naver.com/nyblog?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=15424483%26search.menuid=251%26search.searchdate=2021-07-012021-09-30%26search.searchBy=1%26search.sortBy=date%26search.option=0%26userDisplay=50%26search.query=%BB%FE%B3%DA%26search.includeAll=%26search.exclude=%26search.include=%26search.exact=%26search.page={}".format(page)
    return search_url

data = [] 

for page in range(1):
    url = move_page(page)
    driver.get(url)
    driver.switch_to.frame('cafe_main')
    search_url = driver.page_source
    soup = BeautifulSoup(search_url, 'html.parser')
    articles = soup.select('div.inner_list')

    for article in articles:
        subject = article.find('a', class_='article')

        if subject is None:
            continue

        article_url = 'https://cafe.naver.com' + subject['href']
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])
        driver.get(article_url)
        time.sleep(3)  
        driver.switch_to.frame('cafe_main')

        article_page_source = driver.page_source
        soup = BeautifulSoup(article_page_source, 'html.parser')
        shoppingMall = soup.select_one('.se-module.se-module-text p:nth-of-type(1)')
        goods = soup.select_one('.se-module.se-module-text p:nth-of-type(3)')
        price_tag = soup.select_one('.se-module.se-module-text p:nth-of-type(4)')

        if price_tag is not None and shoppingMall is not None and goods is not None:
            shoppingMall = shoppingMall.text.strip()
            shoppingMall = shoppingMall.split('】')[-1].strip()
            goods = goods.text.strip()
            goods = goods.split('】')[-1].strip()
            price_text = price_tag.text.strip()  
            price = price_text.split(' ')[-1]  
            data.append(shoppingMall+' , '+goods+ ' , ' +price)

        driver.close()
        driver.switch_to.window(driver.window_handles[0])
    time.sleep(2)
    
with open('21_3_CHANEL_F.txt', 'w', encoding='utf-8') as file:
    for line in data:
        for l in line:
            file.write(l)
        file.write('\n')

KeyboardInterrupt: 